In [2]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt


--2023-10-03 16:32:39--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.001s  

2023-10-03 16:32:39 (22.1 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2023-10-03 16:32:39--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP re

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [4]:
input_files = ['edgar_allan_poe.txt','robert_frost.txt']

In [5]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [6]:
input_text=[]
labels=[]


for label,f in enumerate(input_files): #enumarate gives a index to each thing in list here 0 for first file and 1 for other poem
  for line in open(f): #label is index of file and f is the file .txt format
    line=line.rstrip().lower() #removing \n from last and converting into lower case
    if line: # there could be empty lines in the text b/w paragraphs
      line = line.translate(str.maketrans('','',string.punctuation))
      input_text.append(line)
      labels.append(label)

In [7]:
tr_text,tt_text,tr_labels,tt_labels=train_test_split(input_text,labels)

In [8]:
idx=1
word2idx={'<unk>':0}

for text in tr_text:
  words=text.split() #could use complex things like stemming or lemmatizer
  for word in words:
    if word not in word2idx:
      word2idx[word]=idx
      idx+=1


In [9]:
tr_text_int=[]
tt_text_int=[]

In [10]:

for text in tr_text:
  words=text.split()
  line_as_int=[word2idx[word] for word in words]
  tr_text_int.append(line_as_int)



In [11]:

for text in tt_text:
  words=text.split()
  line_as_int=[word2idx.get(word,0) for word in words]
  tt_text_int.append(line_as_int)

In [12]:
V =len(word2idx)
A0 =np.ones((V,V)) #corresponfding to first author
pi0=np.ones(V)

A1=np.ones((V,V))
pi1=np.ones(V)
# therfore we create 2 markov models for each poer/

In [13]:

def compute(text_as_int , A, pi):
  for tokens in text_as_int :
    last_idx= None
    for idx in tokens:
      if last_idx is None:
        pi[idx]+=1
      else:
        A[last_idx,idx]+=1
      last_idx=idx

compute([t for t,y in zip(tr_text_int,tr_labels) if y==0] , A0,pi0)
compute([t for t,y in zip(tr_text_int,tr_labels) if y==1] , A1,pi1)

In [14]:
 #converting counts to probailites
 A0 /= A0.sum(axis=1,keepdims=True)
 pi0 /=pi0.sum()
 A1 /= A1.sum(axis=1,keepdims=True)
 pi1 /=pi1.sum()

In [15]:
logA0=np.log(A0)
logpi0=np.log(pi0)

logA1=np.log(A1)
logpi1=np.log(pi1)

In [16]:
count0=0
count1=0
for y in tr_labels:
  if y==0:
    count0+=1
  else:
    count1+=1
tot=len(tr_labels)
p0=count0/tot
p1=count1/tot
logp0=np.log(p0)
logp1=np.log(p1)
p0,p1


(0.33250773993808047, 0.6674922600619195)

In [33]:

class Classifier:
  def __init__(self,logAs,logpis,logpriors):
    self.logAs=logAs
    self.logpis=logpis
    self.logpriors=logpriors
    self.k=len(logAs)


  def compute_log_likelihood(self,input_,class_):
    logA=self.logAs[class_]
    logpi=self.logpis[class_]

    last_idx=None

    logprob=0

    for idx in input_:
      if last_idx==None:
        logprob+=logpi[idx]
      else :
        logprob+=logA[last_idx,idx]
      last_idx=idx

    return logprob

  def predict(self,inputs):
    predictions=np.zeros(len(inputs))
    for i,input_ in enumerate(inputs):
      post_=[self.compute_log_likelihood(input_,c)+self.logpriors[c] for c in range(self.k)]
      pred=np.argmax(post_)
      predictions[i]=pred
    return predictions




In [34]:
clf=Classifier([logA0,logA1],[logpi0,logpi1],[logp0,logp1])

In [35]:
P_train = clf.predict(tr_text_int)
print(f"Train accuracy: {np.mean(P_train==tr_labels)}")

Train accuracy: 0.993188854489164


In [36]:
P_test = clf.predict(tt_text_int)
print(f"Test accuracy: {np.mean(P_test==tt_labels)}")

Test accuracy: 0.8033395176252319


In [37]:
from sklearn.metrics import confusion_matrix,f1_score


In [38]:
cm_train=confusion_matrix(tr_labels,P_train)
cm_train

array([[ 526,   11],
       [   0, 1078]])

In [39]:
cm_test=confusion_matrix(tt_labels,P_test)
cm_test

array([[ 83,  98],
       [  8, 350]])

In [40]:
f1_score(tr_labels,P_train)

0.9949238578680203

In [41]:
f1_score(tt_labels,P_test)

0.8684863523573202